# 装饰器

装饰器就是封装一个函数，在不修改原函数的形况下，给原函数增加新的功能。  
> 封装函数就是在一个函数里面，调用另一个函数。    

与平常函数调用的区别就是，这个**封装后的函数**与**原函数**的外形是一样的。  

假设有一个原函数的功能是，一秒后打印 "Hello World!"。

In [3]:
import time
def printDelay():
    time.sleep(1)
    print("Hello World!")
printDelay()

Hello World!


现在需要一个装饰器，能够计算原函数的执行时间。
由上面可知，封装函数需要模仿原函数的外形，来取代原函数。
所以封装函数如下定义：
```python
def warpper(*args, **kw):
    start = time.time()
    ret = fun(*args, **kw)
    print(fun.__name__, "run time:", time.time() - start)
    return ret
```
这里 `*args, **kw` 表示任意入参，`fun` 为需要封装的原函数。

而**装饰器**就是要把这个**封装后的函数**作为变量返回，以达到取代原函数的作用。

In [5]:
def printRunTime(fun): # 装饰器函数的入参就是要封装的原函数
    def warpper(*args, **kw): # 将上面带具体功能的封装函数复制进来
        start = time.time()
        ret = fun(*args, **kw)
        print(fun.__name__, "run time:", time.time() - start)
        return ret
    return warpper # 返回值就是封装后的函数

注意，装饰器只能用在函数定义的地方。

In [8]:
@printRunTime
def delayPrint(t):
    time.sleep(t)
    print("Hello World!")

delayPrint(1)

Hello World!
delayPrint run time: 1.0030255317687988


这样一个简单的计算函数运行时间的装饰器就写好了。  
以上面为例：装饰器的运行逻辑就是 `delayPrint = printRunTime(delayPrint)`

## 装饰器的入参
如果装饰器本身需要入参呢？上面的装饰器函数是没有入参的。
现在改造上面的 printRunTime 装饰器，添加在函数执行前输出自定义的文本。  
首先确定封装函数怎么写。
```python
def warpper(*args, **kw): 
    print(text)    # 新增的功能
    start = time.time()
    ret = fun(*args, **kw)
    print(fun.__name__, "run time:", time.time() - start)
    return ret
```
这里有两个未知的变量，  
`text` : 自定义的打印的文本。  
`fun`  : 要被封装的原函数。  
所以需要套两层函数，才能实现替换功能。每层函数的入参只有一个。  

In [13]:
def printRunTimeLog(text):
    def Decorator(fun):
        def warpper(*args, **kw): # 将上面带具体功能的封装函数复制进来
            print(text)    # 新增的功能
            start = time.time()
            ret = fun(*args, **kw)
            print(fun.__name__, "run time:", time.time() - start)
            return ret
        return warpper
    return Decorator

@printRunTimeLog("开始运行了...")
def delayPrint1(t):
    time.sleep(t)
    print("Hello World!")

delayPrint1(2)   

开始运行了...
Hello World!
delayPrint1 run time: 2.0121119022369385


为什么装饰器入参明明没有原函数的位置，装饰器函数却能准确的装饰上原函数呢？  
只要理解装饰器运行原理，就很好理解了。  
`delayPrint1 = printRunTimeLog("开始运行了...")(delayPrint1)`  
因为装饰器 `printRunTimeLog` 返回值就是一个函数（`Decorator`），而这个函数的入参就是 `delayPrint1`。

## warps()

In [16]:
print(delayPrint1.__name__)

warpper


执行上面代码发现，被上面的装饰器装饰后，函数的`__name__` 属性变了。这很好理解，因为原函数已经被 warrper 函数给取代了。
那么，怎么恢复原来的函数属性呢？
稍加修改装饰器函数即可：

In [22]:
from functools import wraps

def printRunTimeLog(text):
    def Decorator(fun):
        @wraps(fun) # 在封装函数之前添加一个装饰器即可（具体细节不究）
        def warpper(*args, **kw):
            print(text)
            start = time.time()
            ret = fun(*args, **kw)
            print(fun.__name__, "run time:", time.time() - start)
            return ret
        return warpper
    return Decorator

@printRunTimeLog("开始运行了...")
def delayPrint1(t):
    time.sleep(t)
    print("Hello World!")
    
# delayPrint1(2)
print(delayPrint1.__name__)

delayPrint1


## 类装饰器
作用，概念都什么区别。只是写法变了，之前是装饰器函数，现在是装饰器类。
以下仅举例。

In [23]:
import time

class A:
    def __init__(self, func): # 原函数func
        self._func = func
    
    def __call__(self, *args, **kwargs): # 封装扩展功能
        start = time.time()
        result=self._func(*args, **kwargs)  # 原来功能
        end = time.time()
        print("总共耗时{:.4} s".format(end - start))
        return result

@A
def count_prime_number(maxnumber):
    time.sleep(1)
    sum=0
    for i in range(2, maxnumber):
            sum=sum+i
    return sum

sums=count_prime_number(10000)
print(sums)

总共耗时1.002 s
49994999
